In [29]:
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import numpy as np
from tqdm.notebook import tqdm

In [2]:
out_folder = "outs/o_o"
method_folders = glob(f"{out_folder}/*")

In [3]:
print(method_folders)

['outs/o_o/eigencam', 'outs/o_o/gradcam', 'outs/o_o/xgradcam', 'outs/o_o/gradcampp', 'outs/o_o/fullgrad']


In [4]:
method_ims = defaultdict(dict)
for method_folder in method_folders:
    method = method_folder.split("/")[-1]
    for image_fn in glob(f"{method_folder}/*.*"):
        bn = os.path.basename(image_fn)
        method_ims[method][bn] = np.array(Image.open(image_fn))

In [40]:
method_diffs = defaultdict(float)
for bn in tqdm(list(method_ims["gradcam"].keys())):
    current_ims = {}
    for method in method_ims:
        current_ims[f"{method}_{bn}"] = method_ims[method][bn]
    
    pairs = []
    for im_f, im in current_ims.items():
        for im2_f, im2 in current_ims.items():
            for pair in pairs:
                if im_f in pair and im2_f in pair:
                    break
            else:
                pairs.append((im_f, im2_f))
                im, im2 = np.array(im, dtype='float32'), np.array(im2, dtype='float32')
                # print(f"{im_f} - {im2_f}", np.sum(np.abs(im - im2)))
                difference = np.sum(np.abs(im - im2))
                method_diffs[im_f.split('_')[0]] += difference
                method_diffs[im2_f.split('_')[0]] += difference             

  0%|          | 0/4577 [00:00<?, ?it/s]

In [41]:
method_diffs = dict(sorted(method_diffs.items(), key=lambda x: x[1], reverse=True))
for k, v in method_diffs.items():
    print(f"{k}: {v}")

fullgrad: 93880160989.0
gradcampp: 81532256940.0
xgradcam: 81326763854.0
gradcam: 80688737703.0
eigencam: 78654802946.0
